<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: calwebb_image3 astrometry: tweakreg and source_catalog with MIRI using Gaia

<span style="color:red"> **Instruments Affected**</span>: MIRI, NIRCam 


Tested on MIRI Commissioning data

### Table of Contents
<div style="text-align: left"> 

<br>  [Introduction](#intro_ID) <br> [Test Description](#Test_descrip) <br> [Run Pipeline](#pipeline_ID) <br> [Check Results](#output_ID) <br> [About This Notebook](#about_ID) <br>


</div>

<a id="intro_ID"></a>

## Introduction

This notebook processes a set of images through calwebb_detector1, calwebb_image2 and calwebb_image3 and examines the output table of the source_catalog step along with comparisons to Gaia (aligning the data to Gaia in tweakreg).

This test uses MIRI F770W Imager data of a crowded star field in the LMC. The data is from Commissioning program 1040. Longer wavelength data using F1280W filter from the same program are also compared to Gaia and the F770W data. 

The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/jwst/source_catalog/main.html

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

<a id="Test_descrip"></a>
## Test description


The steps of this test are as follow:

For each filter,

1) Set up data path and directory and image files name.

2) Run output of calwebb_detector1 through calwebb_image2.

3) Run output of calwebb_image2 through calwebb_image3. Use Align_to_Gaia in image3.

4) Read in output table of source_catalog step and print ecsv table for pipeline output and gaia catalog.

5) Display image and overplot detector sources from ecsv table, comparing pipeline and gaia catalogs.

6) Look at plots of total flux in Jy and AB mag.

7) Look for matches between expected source positions (RA and Dec) from Gaia catalog to output from source_catalog.

8) Examine how far apart any matched sources are between Gaia and source catalog.

9) Compare magnitudes and magnitude differences between the different filters and output found sources.


### Create a temporary directory for all of the test data

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)

# For info, print out where the script is running
print("Running in {}".format(os.getcwd()))

Running in /internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpiafa16mu


In [2]:
import os
if 'CRDS_CACHE_TYPE' in os.environ:
    if os.environ['CRDS_CACHE_TYPE'] == 'local':
        os.environ['CRDS_PATH'] = os.path.join(os.environ['HOME'], 'crds', 'cache')
    elif os.path.isdir(os.environ['CRDS_CACHE_TYPE']):
        os.environ['CRDS_PATH'] = os.environ['CRDS_CACHE_TYPE']
print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

CRDS cache location: /tmp/crds_cache


### Set up import statements

In [3]:
#import pytest
import numpy as np
from glob import glob
import json
import matplotlib.pyplot as plt
import photutils
import math

from astropy.io import fits, ascii
from astropy.coordinates import Angle
from astropy.table import Table, vstack, unique, join
from astropy import table
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.visualization import simple_norm
from astropy import units as u
from astropy.modeling import models
from astropy.wcs import WCS

# Box download imports 
from astropy.utils.data import download_file
from pathlib import Path
from shutil import move
from os.path import splitext

import jwst
from jwst import datamodels
from jwst.datamodels import RampModel, ImageModel

from jwst import associations
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

from jwst.pipeline import calwebb_image3
from jwst.pipeline import calwebb_image2
from jwst.pipeline import calwebb_detector1
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline

### Print pipeline version

In [4]:
print(jwst.__version__) 
print(data_dir)

1.7.2
<TemporaryDirectory '/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpiafa16mu'>


### Read in data

In [5]:
# Read in dataset from Box

def get_box_files(file_list):
    for box_url,file_name in file_list:
        if 'https' not in box_url:
            box_url = 'https://stsci.box.com/shared/static/' + box_url
        downloaded_file = download_file(box_url, timeout=600)
        if Path(file_name).suffix == '':
            ext = splitext(box_url)[1]
            file_name += ext
        move(downloaded_file, file_name)
 
# F770W data of PID 1040 (LMC), taken in May 2022   
file_urls = ['https://stsci.box.com/shared/static/yk2kfwxr1dv7mcc84zvi02t7mq8e5wc7.fits',
             'https://stsci.box.com/shared/static/icedk709owq6op5mttv4y8k0bce58wu7.fits',
             'https://stsci.box.com/shared/static/kq4h4pb95yse3k1ryb0d1vcs69ay4679.fits',
             'https://stsci.box.com/shared/static/z1wbjh0y3dzuh2i5heejo0sehwqu2p2w.fits',
             'https://stsci.box.com/shared/static/q01wh0evq7w7klqy566cbm1qlpuys7h7.fits',
             'https://stsci.box.com/shared/static/gj558uxvarmcm807z7hv3h1nh7hscudc.fit'] 

file_names = ['jw01040001005_03103_00001_mirimage_uncal.fits',
              'jw01040001005_03103_00002_mirimage_uncal.fits',
              'jw01040001005_03103_00003_mirimage_uncal.fits',
              'jw01040001005_03103_00004_mirimage_uncal.fits',
              'jw01040001005_03103_00005_mirimage_uncal.fits',
              'Gaia_pid_1040.fit']

In [6]:
box_download_list = [(url,name) for url,name in zip(file_urls,file_names)]


get_box_files(box_download_list)

In [7]:
uncalfiles = file_names[0:-1]

full_gaia_cat = file_names[-1]

print('Filenames of uncal files')
print(uncalfiles)
print()
print('Gaia catalog name: ', full_gaia_cat)

Filenames of uncal files
['jw01040001005_03103_00001_mirimage_uncal.fits', 'jw01040001005_03103_00002_mirimage_uncal.fits', 'jw01040001005_03103_00003_mirimage_uncal.fits', 'jw01040001005_03103_00004_mirimage_uncal.fits', 'jw01040001005_03103_00005_mirimage_uncal.fits']

Gaia catalog name:  Gaia_pid_1040.fit


<a id="pipeline_ID"></a>
### Run Pipelines 

Run uncal files through calwebb_detector1, then the rate files through calwebb_image2.

In [8]:
# Run Calwebb_detector1 on uncal.fits files
#uncalfiles = glob('*uncal.fits')    
    
print('There are ', len(uncalfiles), ' images.')
        
slopelist = []    
    
# loop over list of files
for file in uncalfiles:

    # Run pipeline on each file
    rampfile = Detector1Pipeline.call(file, save_results=True)
    
    # set up output file name
    base, remainder = file.split('.')
    outname = base
    
    slopelist.append(base+'rate.fits')

print('Detector 1 steps completed on all files.')
print(slopelist)   
    

There are  5  images.


2022-09-20 12:34:19,708 - CRDS - INFO -  Fetching  /tmp/crds_cache/mappings/jwst/jwst_nirspec_superbias_0051.rmap   26.4 K bytes  (1 / 13 files) (0 / 124.3 K bytes)


Traceback (most recent call last):
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 347, in local_bestrefs
    return hv_best_references(context, parameters, reftypes)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 367, in hv_best_references
    ctx = get_symbolic_mapping(context_file, cached=True)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 670, in get_symbolic_mapping
    return get_pickled_mapping(   # reviewed
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks

Traceback (most recent call last):
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 347, in local_bestrefs
    return hv_best_references(context, parameters, reftypes)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 367, in hv_best_references
    ctx = get_symbolic_mapping(context_file, cached=True)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 670, in get_symbolic_mapping
    return get_pickled_mapping(   # reviewed
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks

Traceback (most recent call last):
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 347, in local_bestrefs
    return hv_best_references(context, parameters, reftypes)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 367, in hv_best_references
    ctx = get_symbolic_mapping(context_file, cached=True)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 670, in get_symbolic_mapping
    return get_pickled_mapping(   # reviewed
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks

...

2022-09-20 12:34:19,971 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_calibrated_ramp': False, 'steps': {'group_scale': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}, 'dq_init': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}, 'saturation': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'outpu

2022-09-20 12:34:20,199 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'jw01040001005_03103_00001_mirimage_uncal.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']


2022-09-20 12:34:20,205 - CRDS - INFO -  Fetching  /tmp/crds_cache/mappings/jwst/jwst_nirspec_superbias_0051.rmap   26.4 K bytes  (1 / 13 files) (0 / 124.3 K bytes)


Traceback (most recent call last):
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 347, in local_bestrefs
    return hv_best_references(context, parameters, reftypes)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 367, in hv_best_references
    ctx = get_symbolic_mapping(context_file, cached=True)
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks/lib/python3.9/site-packages/crds/core/heavy_client.py", line 670, in get_symbolic_mapping
    return get_pickled_mapping(   # reviewed
  File "/internal/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/miniconda3/envs/jwst_validation_notebooks

CrdsDownloadError: Failed caching mapping files: Error fetching data for 'jwst_nirspec_superbias_0051.rmap' at CRDS server 'https://jwst-crds.stsci.edu' with mode 'http' : [Errno 28] No space left on device: '/tmp/crds_cache/mappings/jwst/jwst_nirspec_superbias_0051.rmap'

In [ ]:
# Run Calwebb_image2 on output files from detector1
ratefiles = glob('*rate.fits')    
    
print('There are ', len(ratefiles), ' images.')
    
callist = []

# cycle through files
for im in ratefiles:

    calfile = Image2Pipeline.call(im, save_results=True)

    callist.append(calfile)

print(callist)

### Create association file to run through Calwebb_image3

In [ ]:
# use asn_from_list to create association table

calfiles = glob('*_cal.fits')
asn = asn_from_list.asn_from_list(calfiles, rule=DMS_Level3_Base, product_name='starfield_combined.fits')

# dump association table to a .json file for use in image3
with open('starfield_asnfile.json', 'w') as fp:
    fp.write(asn.dump()[1])

print(asn) 

### Run Calwebb_Image3 pipeline
For MIRI, the FWHM values are dependent on filter and should be set using the table below:

|Filter | FWHM  |
|-------| -------|
|F560W      |     1.636		|	
|F770W      |     2.187		|
|F1000W     |     2.888		|
|F1130W     |     3.318		|
|F1280W     |     3.713		|
|F1500W     |     4.354		|
|F1800W     |     5.224		|
|F2100W     |     5.989		|
|F2550W     |     7.312		|
|F2550WR    |     7.312	    | 

For the fit geometry keyword, the following options are available:
fitgeometry: A str value indicating the type of affine transformation to be considered when fitting catalogs. Allowed values:

  *  'shift': x/y shifts only

  *  'rscale': rotation and scale

  *  'rshift': rotation and shifts

  *  'general': shift, rotation, and scale (Default=”general”)

In [ ]:
# Run Calwebb_image3 on the association table
    
# set any specific parameters

# Set tweakreg parameters:

# tweakreg parameters to allow data to run
fwhm = 2.187  # Gaussian kernel FWHM of objects expected, default=2.5
snr = 5 # signal to noise threshold, default=5 
sigma = 1.5 # clipping limit, in sigma units, used when performing fit, default=3
minobj = 5 # number of objects needed to match
fit_geom ='shift' # ftype of affine transformation to be considered when fitting catalogs, default='general'
search_radius = 1.0 # radius in arcseconds to search for a match
tol = 0.7  # Matching tolerance for xyxymatch in arcsec. (Default=1.0)
use2dhist = True  # boolean indicating whether to use 2D histogram to find initial offset, default=True
gaia = True
gaia_ver = 'GAIADR2'
min_gaia = 3
save_gaia = True
deblend = False
npixels = 5
   
pipe3=Image3Pipeline()    
pipe3.tweakreg.kernel_fwhm = fwhm
pipe3.tweakreg.snr_threshold = snr
pipe3.tweakreg.minobj = minobj
pipe3.tweakreg.sigma = sigma
pipe3.tweakreg.fitgeometry = fit_geom
pipe3.tweakreg.use2dhist = use2dhist
pipe3.tweakreg.save_catalogs = True
pipe3.tweakreg.searchrad = search_radius
pipe3.tweakreg.tolerance = tol
pipe3.tweakreg.align_to_gaia = gaia
pipe3.tweakreg.gaia_catalog = gaia_ver
pipe3.tweakreg.min_gaia = min_gaia
pipe3.tweakreg.save_gaia_catalog = save_gaia

pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm
pipe3.source_catalog.deblend = deblend
pipe3.source_catalog.npixels = npixels
pipe3.save_results = True

# run Image3

image = pipe3.run('starfield_asnfile.json')
print('Image 3 pipeline finished.')

<a id="output_ID"></a>
## Results
Read in the output of the pipeline and check your results.

### Read in output table of source_catalog step and print ecsv table

In [ ]:
photfile = 'starfield_combined_cat.ecsv'
input_file = 'starfield_combined_i2d.fits'

In [ ]:
# Look at catalog table that shows all columns, but subset of rows
# Pay attention to rows with a large number of nans, as this may indicate a spurious source
cat_data = table.Table.read(photfile, format='ascii', comment='#')

catalog = Table.read(photfile)
miri_x = catalog['xcentroid']
miri_y = catalog['ycentroid']
catalog

In [ ]:
# Look at output gaia catalog
gaia_out = 'fit_gaiadr2_ref.ecsv'

In [ ]:
gaia_cat = table.Table.read(gaia_out, format='ascii', comment='#')
gaia_cat

### Display image and overplot detector sources from ecsv table 

In [ ]:
# Read in i2d combined Image
im_i2d = ImageModel(input_file)   

In [ ]:
# read in ecsv photom file output from source catalog
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval
from astropy import table

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d.data, origin='lower', cmap='rainbow', vmin=3, vmax=6)
plt.colorbar()
plt.scatter(miri_x, miri_y,lw=1, s=10,color='black')

In [ ]:
# Get from RA and Dec in Gaia catalog to x and y positions on image

world_to_detector = im_i2d.meta.wcs.get_transform('world', 'detector')
xgaia,ygaia = world_to_detector(gaia_cat['RA'], gaia_cat['DEC'])

In [ ]:
# read in ecsv gaia photom file output from source catalog
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval
from astropy import table

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d.data, origin='lower', cmap='rainbow', vmin=0, vmax=7)
plt.colorbar()
plt.scatter(xgaia, ygaia,lw=1, s=10,color='black')

### Double-check pipeline Gaia catalog against catalog from Vizier (Gaia DR2)

In [ ]:
# Read in Gaia catalog from Vizier

gaia_hdu = fits.open(full_gaia_cat) #('gaia_tweakreg_test3.fit')
gaia_data = gaia_hdu[1].data

# Get RA and Dec of Gaia sources
gaia_ra = gaia_data['RA_ICRS']
gaia_dec = gaia_data['DE_ICRS']

xgaia_full,ygaia_full = world_to_detector(gaia_ra, gaia_dec)

gaia_hdu.close()

In [ ]:
# read in ecsv gaia photom file output from source catalog
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval
from astropy import table

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d.data, origin='lower', cmap='rainbow', vmin=0, vmax=7)
plt.colorbar()
plt.scatter(xgaia_full, ygaia_full,lw=1, s=10,color='black')

### Compare Source catalog positions to Gaia positions found by pipeline

In [ ]:
# Many sources go beyond image edges. Only display sources that were actually in FOV of combined image.
minval = 0
maxval = 1200

ind = np.where((xgaia > minval) & (xgaia < 1110)& (ygaia > minval) & (ygaia < maxval))

In [ ]:
plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d.data, origin='lower', cmap='rainbow', vmin=0, vmax=7)
plt.colorbar()
plt.scatter(miri_x, miri_y,lw=1, s=10,color='black')
plt.scatter(xgaia[ind], ygaia[ind],lw=1, s=5,color='white')

In [ ]:
# Show zoomed in region to see if stars look like point sources and aren't smeared out or doubled

xmin = 700
xmax = 900
ymin = 500
ymax = 700

gaiazoom = np.where((xgaia > xmin) & (xgaia < xmax) & (ygaia > ymin) & (ygaia < ymax))
print(gaiazoom)
subx = xgaia[gaiazoom] - xmin
suby = ygaia[gaiazoom] - ymin

mirizoom = np.where((miri_x > xmin) & (miri_x < xmax) & (miri_y > ymin) & (miri_y < ymax))
print(gaiazoom)
subxmiri = miri_x[mirizoom] - xmin
subymiri = miri_y[mirizoom] - ymin


plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d.data[ymin:ymax,xmin:xmax], origin='lower', cmap='rainbow', vmin=0, vmax=7)

plt.scatter(subx, suby,lw=1, s=10,color='white')
plt.scatter(subxmiri, subymiri,lw=1, s=5,color='black')
plt.colorbar()

print('Gaia sources are marked in white, MIRI sources from the pipeline in black')

In [ ]:
# Calculate difference in position between catalog and Gaia positions. There is an offset here.

tol = 1    # Do calculations in pixels
found_count=0

# Set up array for matches
diffarr = np.array(len(xgaia))
      
deltax = []
deltay = []
xpos = []
ypos = []

print('         x             y           xdiff          ydiff    < 0.5 pix')
for i in np.arange(0,len(xgaia)):
    for j in np.arange(0,len(miri_x)):
        x_diff = abs(xgaia[i] - miri_x[j])
        y_diff = abs(ygaia[i] - miri_y[j])

        if x_diff < tol and y_diff < tol:
            deltax.append(xgaia[i] - miri_x[j])
            deltay.append(ygaia[i] - miri_y[j])
            xpos.append(miri_x[j])
            ypos.append(miri_y[j])
            found_count +=1

            if (x_diff < 0.5) and (y_diff < 0.5): 
                test = 'pass' 
            else: 
                test = 'fail'
            print('{:15.6f} {:15.6f} {:15.6f} {:15.6f} {}'.format(miri_x[j], miri_y[j], x_diff, y_diff, test))

print()            
print(found_count,' matches found')
diffarr = np.zeros(found_count)
for match in np.arange(0,found_count):            
    diffarr[match] = math.sqrt((deltax[match]**2)+(deltay[match]**2))
print()
#print(diffarr)

In [ ]:
# Look at differences in position
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(deltax, deltay)
plt.title('position offset between catalog and Gaia pos')
plt.xlabel('x difference in pixels')
plt.ylabel('y difference in pixels')

In [ ]:
# Look at differences in position
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(xpos, diffarr)
plt.title('position offset between catalog and Gaia pos')
plt.xlabel('x position in pixels')
plt.ylabel('magnitude of vector diff in pixels')

In [ ]:
# Look at differences in position
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(ypos, diffarr)
plt.title('position offset between catalog and Gaia pos')
plt.xlabel('y position in pixels')
plt.ylabel('magnitude of vector diff in pixels')

## Compare RA, Dec positions in catalogs 

In [ ]:
# Compare Gaia catalogs to see if the pipeline used the same dataset out of Vizier
# set the tolerance for differences and initialize counters

tol = 1.e-5    # Set tolerance around 30 mas (units here are in degrees; 36 mas ~ 1e-5 deg)
found_count=0
multiples_count=0
missed_count=0

# Set up array for matches
detected = np.chararray(len(gaia_ra))
#print(np.shape(detected))

for ra,dec,idx in zip(gaia_ra, gaia_dec,range(len(gaia_ra))):

    match = np.where((np.abs(gaia_cat['RA']-ra) < tol) & (np.abs(gaia_cat['DEC']-dec) < tol))
    #print('match', match)
    
    if np.size(match) == 1: 
        found_count +=1 
        detected[idx] = 'Y'

    if np.size(match) > 1:  
        multiples_count +=1       
        
    if np.size(match) < 1:
        missed_count +=1

In [ ]:

total_percent_found = (found_count/len(gaia_ra))*100

print('\n')
print('SNR threshold used for pipeline: ',pipe3.source_catalog.snr_threshold)
print('Total matches found:',found_count)
print('Total missed:',missed_count)
print('Number of multiples: ',multiples_count)
print('Total number of input (gaia) sources:',len(gaia_ra))
print('Total number in output (gaia) catalog:',len(gaia_cat))
print('Total percent found:',total_percent_found)
print('\n')

if (found_count == len(gaia_cat)):
    print('All gaia images output by pipeline match Gaia sources found from Vizier')

### Compare number of sources found by pipeline to number of gaia sources in pipeline output catalog

In [ ]:
# Get RA and Dec for sources found in output catalog by pipeline
cat_ra = catalog['sky_centroid'].ra.deg
for i in range(len(cat_ra)):
    if (cat_ra[i]>180.):
        cat_ra[i] -= 360.
cat_dec = catalog['sky_centroid'].dec.deg

#print(cat_ra)
#print()
#print(cat_dec)

In [ ]:
# Look at position differences in RA and Dec rather than pixels

allRAdiff_cal = []
allDecdiff_cal = [] 
    
# Get coordinates with SkyCoord for each catalog
cat_miri = Table([cat_ra, cat_dec], names=('ra', 'dec'))
cat_gaia = Table([gaia_cat['RA'], gaia_cat['DEC']], names=('ra', 'dec'))
    
coord_cat_gaia = SkyCoord(ra=cat_gaia['ra'], dec=cat_gaia['dec'], unit="deg")
coord_cat_miri = SkyCoord(ra=cat_miri['ra'], dec=cat_miri['dec'], unit="deg")
ind_catmiri_catgaia, dist_2d, _ = match_coordinates_sky(coord_cat_gaia, coord_cat_miri)
    
# Find where the catalogs match
    
cat1_matched = cat_gaia[dist_2d.arcsec<0.05]
cat2_matched = cat_miri[ind_catmiri_catgaia[dist_2d.arcsec<0.05]]
    
#print(cat1_matched)
    
# Get differences in RA, Dec
ra_diff = cat2_matched['ra'] - cat1_matched['ra']
dec_diff = cat2_matched['dec'] - cat1_matched['dec'] 
    
#print(ra_diff)
    
# put differences in milliarcseconds
ra_diff = ra_diff * 3600000
dec_diff = dec_diff * 3600000
    
    
# Compare input RA, Dec to found RA, Dec
print('RA_Diff (mas)  Dec_diff (mas) pass/fail')
# Find if the differences are within the allowed 30 mas range
for i in np.arange(0,len(ra_diff)):
    #if ra_diff[i] < 30 and dec_diff[i] < 30:
    allRAdiff_cal.append(ra_diff[i])
    allDecdiff_cal.append(dec_diff[i])
    if abs(ra_diff[i]) < 30 and abs(dec_diff[i]) < 30: 
        test = 'pass' 
    else: 
        test = 'fail'
    print('{:15.6f} {:15.6f} {}'.format(ra_diff[i], dec_diff[i], test))
 
    
# Plot ra and dec differences
plt.title ('Differences in RA and Dec in milliarcseconds')
plt.ylabel('Delta RA (mas)')
plt.xlabel('Delta Dec (mas)')
plt.scatter(ra_diff,dec_diff)
plt.show()

In [ ]:
# Compare Gaia catalogs to see if the pipeline used the same dataset out of Vizier
# set the tolerance for differences and initialize counters

tol = 1.e-5    # Set tolerance around 30 mas (units here are in degrees; 36 mas ~ 1e-5 deg)
found_count=0
multiples_count=0
missed_count=0

# Set up array for matches
detected = np.chararray(len(gaia_ra))
#print(np.shape(detected))

for ra,dec,idx in zip(gaia_cat['RA'], gaia_cat['DEC'],range(len(gaia_cat['RA']))):

    match = np.where((np.abs(cat_ra-ra) < tol) & (np.abs(cat_dec-dec) < tol))
    #print('match', match)
    
    if np.size(match) == 1: 
        found_count +=1 
        detected[idx] = 'Y'

    if np.size(match) > 1:  
        multiples_count +=1       
        
    if np.size(match) < 1:
        missed_count +=1

In [ ]:
total_percent_found = (found_count/len(gaia_cat['RA']))*100

print('\n')
print('SNR threshold used for pipeline: ',pipe3.source_catalog.snr_threshold)
print('Total matches found:',found_count)
#print('Total missed:',missed_count)
print('Number of multiples: ',multiples_count)
print('Total number of input (Gaia) sources:',len(gaia_cat['RA']))
print('Total number in output JWST catalog:',len(cat_ra))
print('Total percent found:',total_percent_found)
print('\n')

## Look at results of photometry with catalog

In [ ]:
# Look at flux
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(catalog['label'], catalog['aper_total_flux'])
plt.yscale('log')
plt.title('Total Flux in '+ str(catalog['aper_total_flux'].unit))
plt.xlabel('label')
plt.ylabel('aper_total_flux')

In [ ]:
# See the relationship between Flux and error
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

maxflux = 0.0012

index = np.where(catalog['aper_total_flux'] < maxflux)

ax.scatter(catalog['aper_total_flux'][index], catalog['aper_total_flux_err'][index])
#plt.yscale('log')
plt.title('Total Flux vs. Flux err')
plt.xlabel('Flux in Jy')
plt.ylabel('Flux err')

In [ ]:
# Look at AB magnitudes
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(catalog['label'], catalog['aper_total_abmag'])
#plt.yscale('log')
plt.title('Total AB mag')
plt.xlabel('label')
plt.ylabel('aper_total_abmag')

In [ ]:
# See the relationship between AB mag and error
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(catalog['aper_total_abmag'], catalog['aper_total_abmag_err'])
#plt.yscale('log')
plt.title('Total AB mag vs. AB mag err')
plt.xlabel('AB mag')
plt.ylabel('AB mag err')

## Do the same comparison for longer wavelength data. 
The distortion correction comes from the F770W data. See how well the astrometry holds up at longer wavelengths. The data set below comes from the same program, PID 1040, but includes a set of five images of the same region with the F1280W filter.


### Read in longer wavelength data

In [ ]:
# Read in dataset from Box

def get_box_files(file_list):
    for box_url,file_name in file_list:
        if 'https' not in box_url:
            box_url = 'https://stsci.box.com/shared/static/' + box_url
        downloaded_file = download_file(box_url, timeout=600)
        if Path(file_name).suffix == '':
            ext = splitext(box_url)[1]
            file_name += ext
        move(downloaded_file, file_name)

# F1280W data of PID 1040 (LMC), taken in May 2022   
file_urls = ['https://stsci.box.com/shared/static/kt3zb9b9aynuvtn4ytuxl2yl5bhsbc7l.fits',
             'https://stsci.box.com/shared/static/qebublvwwi1wvsiu6pu6fb6r0vcgf0jz.fits',
             'https://stsci.box.com/shared/static/huuf70x9hg6wnhl1i05hu8dx0fld4yd3.fits',
             'https://stsci.box.com/shared/static/pblxt7pvujuumwrertz94vm5mfcssqv7.fits',
             'https://stsci.box.com/shared/static/5adbhbdrf0nk85cxbhgi11s5lic959ru.fits'] 

file_names_long = ['jw01040001005_03109_00001_mirimage_uncal.fits',
              'jw01040001005_03109_00002_mirimage_uncal.fits',
              'jw01040001005_03109_00003_mirimage_uncal.fits',
              'jw01040001005_03109_00004_mirimage_uncal.fits',
              'jw01040001005_03109_00005_mirimage_uncal.fits']

In [ ]:
box_download_list = [(url,name) for url,name in zip(file_urls,file_names_long)]


get_box_files(box_download_list)

In [ ]:

print('Filenames of uncal files')
print(file_names_long)


### Run data through pipelines

In [ ]:
# Run Calwebb_detector1 on uncal.fits files
#uncalfiles = glob('*uncal.fits')    
    
print('There are ', len(file_names_long), ' images.')
        
slopelist_long = []    
    
# loop over list of files
for file in file_names_long:

    # Run pipeline on each file
    rampfile = Detector1Pipeline.call(file, save_results=True)
    
    # set up output file name
    base_long, remainder = file.split('_uncal')
    outname = base_long
    
    slopelist_long.append(base_long+'_rate.fits')

print('Detector 1 steps completed on all files.')
print(slopelist_long)   

In [ ]:
# Run Calwebb_image2 on output files from detector1
#ratefiles = glob('*rate.fits')    
    
print('There are ', len(slopelist_long), ' images.')
    
callist_long = []

# cycle through files
for im in slopelist_long:

    calfile = Image2Pipeline.call(im, save_results=True)

    callist_long.append(calfile)

print(callist_long)

In [ ]:

calfiles_long = [ele.replace('rate', 'cal') for ele in slopelist_long]
print(calfiles_long)


In [ ]:
# use asn_from_list to create association table

#calfiles_long = glob('base_long*_cal.fits')
asn_long = asn_from_list.asn_from_list(calfiles_long, rule=DMS_Level3_Base, product_name='starfield_combined_long.fits')

# dump association table to a .json file for use in image3
with open('starfield_long_asnfile.json', 'w') as fp:
    fp.write(asn_long.dump()[1])

print(asn_long) 

In [ ]:
# Run Calwebb_image3 on the association table
    
# set any specific parameters

# Set tweakreg parameters:

# tweakreg parameters to allow data to run
fwhm = 4  #3.713  # Gaussian kernel FWHM of objects expected, default=2.5
snr = 3 # signal to noise threshold, default=5 
sigma = 1.5 # clipping limit, in sigma units, used when performing fit, default=3
minobj = 3 # number of objects needed to match
fit_geom ='shift' # ftype of affine transformation to be considered when fitting catalogs, default='general'
search_radius = 1.0 # radius in arcseconds to search for a match
tol = 0.7  # Matching tolerance for xyxymatch in arcsec. (Default=1.0)
use2dhist = True  # boolean indicating whether to use 2D histogram to find initial offset, default=True
gaia = True
gaia_ver = 'GAIADR2'
min_gaia = 2
save_gaia = True
deblend = False
npixels = 5
   
pipe3=Image3Pipeline()    
pipe3.tweakreg.kernel_fwhm = fwhm
pipe3.tweakreg.snr_threshold = snr
pipe3.tweakreg.minobj = minobj
pipe3.tweakreg.sigma = sigma
pipe3.tweakreg.fitgeometry = fit_geom
pipe3.tweakreg.use2dhist = use2dhist
pipe3.tweakreg.save_catalogs = True
pipe3.tweakreg.searchrad = search_radius
pipe3.tweakreg.tolerance = tol
pipe3.tweakreg.align_to_gaia = gaia
pipe3.tweakreg.gaia_catalog = gaia_ver
pipe3.tweakreg.min_gaia = min_gaia
pipe3.tweakreg.save_gaia_catalog = save_gaia

pipe3.source_catalog.save_results = True
pipe3.source_catalog.snr_threshold = snr
pipe3.source_catalog.kernel_fwhm = fwhm
pipe3.source_catalog.deblend = deblend
pipe3.source_catalog.npixels = npixels
pipe3.save_results = True

# run Image3

image = pipe3.run('starfield_long_asnfile.json')
print('Image 3 pipeline finished.')

### Look at results for longer wavelengths

In [ ]:
photfile_long = 'starfield_combined_long_cat.ecsv'
input_file_long = 'starfield_combined_long_i2d.fits'

In [ ]:
# Look at catalog table that shows all columns, but subset of rows
# Pay attention to rows with a large number of nans, as this may indicate a spurious source
#cat_data = table.Table.read(photfile_long, format='ascii', comment='#')

catalog_long = Table.read(photfile_long)
miri_x_long = catalog_long['xcentroid']
miri_y_long = catalog_long['ycentroid']
catalog_long

In [ ]:
# Look at output gaia catalog
gaia_out_long = 'fit_gaiadr2_ref.ecsv'

In [ ]:
gaia_cat = table.Table.read(gaia_out_long, format='ascii', comment='#')
gaia_cat

### Look at image and source catlog overlays

In [ ]:
# Read in i2d combined Image
im_i2d_long = ImageModel(input_file_long)  

In [ ]:
# read in ecsv photom file output from source catalog
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval
from astropy import table

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d_long.data, origin='lower', cmap='rainbow', vmin=18, vmax=26)
plt.colorbar()
plt.scatter(miri_x_long, miri_y_long,lw=1, s=10,color='black')

In [ ]:
# Get from RA and Dec in Gaia catalog to x and y positions on image

world_to_detector = im_i2d_long.meta.wcs.get_transform('world', 'detector')
xgaia_long,ygaia_long = world_to_detector(gaia_cat['RA'], gaia_cat['DEC'])

In [ ]:
# read in ecsv gaia photom file output from source catalog
from astropy.visualization import LogStretch, PercentileInterval, ManualInterval
from astropy import table

#viz2 = LogStretch() + ManualInterval(0,10)

plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d_long.data, origin='lower', cmap='rainbow', vmin=18, vmax=26)
plt.colorbar()
plt.scatter(xgaia_long, ygaia_long,lw=1, s=10,color='black')

### Compare Source catalog positions to Gaia positions found by pipeline
 Start by looking at positions on the image, then calculate any differences in position between Gaia and the output of source_catalog.

In [ ]:
# Many sources go beyond image edges. Only display sources that were actually in FOV of combined image.
minval = 0
maxval = 1200

ind = np.where((xgaia_long > minval) & (xgaia_long < 1110)& (ygaia_long > minval) & (ygaia_long < maxval))

In [ ]:
plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d_long.data, origin='lower', cmap='rainbow', vmin=18, vmax=26)
plt.colorbar()
plt.scatter(miri_x_long, miri_y_long,lw=1, s=10,color='black')
plt.scatter(xgaia_long[ind], ygaia_long[ind],lw=1, s=5,color='white')

In [ ]:
# Show zoomed in region to see if stars look like point sources and aren't smeared out or doubled

xmin = 700
xmax = 900
ymin = 500
ymax = 700

gaiazoom = np.where((xgaia_long > xmin) & (xgaia_long < xmax) & (ygaia_long > ymin) & (ygaia_long < ymax))
print(gaiazoom)
subx = xgaia_long[gaiazoom] - xmin
suby = ygaia_long[gaiazoom] - ymin

mirizoom = np.where((miri_x_long > xmin) & (miri_x_long < xmax) & (miri_y_long > ymin) & (miri_y_long < ymax))
print(gaiazoom)
subxmiri = miri_x_long[mirizoom] - xmin
subymiri = miri_y_long[mirizoom] - ymin


plt.figure(figsize=(20,20))
#plt.imshow(viz2(im_i2d.data), origin='lower')
plt.imshow(im_i2d_long.data[ymin:ymax,xmin:xmax], origin='lower', cmap='rainbow', vmin=18, vmax=26)

plt.scatter(subx, suby,lw=1, s=10,color='white')
plt.scatter(subxmiri, subymiri,lw=1, s=5,color='black')
plt.colorbar()

print('Gaia sources are marked in white, MIRI sources from the pipeline in black')

In [ ]:
# Calculate difference in position between catalog and Gaia positions. There is an offset here.

tol = 1    # Do calculations in pixels
found_count=0

# Set up array for matches
diffarr = np.array(len(xgaia_long))
      
deltax = []
deltay = []
xpos = []
ypos = []

print('         x             y           xdiff          ydiff    < 0.5 pix')
for i in np.arange(0,len(xgaia_long)):
    for j in np.arange(0,len(miri_x_long)):
        x_diff = abs(xgaia_long[i] - miri_x_long[j])
        y_diff = abs(ygaia_long[i] - miri_y_long[j])

        if x_diff < tol and y_diff < tol:
            deltax.append(xgaia_long[i] - miri_x_long[j])
            deltay.append(ygaia_long[i] - miri_y_long[j])
            xpos.append(miri_x_long[j])
            ypos.append(miri_y_long[j])
            found_count +=1

            if (x_diff < 0.5) and (y_diff < 0.5): 
                test = 'pass' 
            else: 
                test = 'fail'
            print('{:15.6f} {:15.6f} {:15.6f} {:15.6f} {}'.format(miri_x_long[j], miri_y_long[j], x_diff, y_diff, test))

print()            
print(found_count,' matches found')
diffarr = np.zeros(found_count)
for match in np.arange(0,found_count):            
    diffarr[match] = math.sqrt((deltax[match]**2)+(deltay[match]**2))
print()

In [ ]:
# Look at differences in position in x and y
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(deltax, deltay)
plt.title('position offset between catalog and Gaia pos')
plt.xlabel('x difference in pixels')
plt.ylabel('y difference in pixels')

In [ ]:
# Look at differences in position (vector magnitude between the two positions against x coordinate)
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(xpos, diffarr)
plt.title('position offset between catalog and Gaia pos')
plt.xlabel('x position in pixels')
plt.ylabel('magnitude of vector diff in pixels')

In [ ]:
# Look at differences in position (vector magnitude between the two positions against y coordinate)
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(ypos, diffarr)
plt.title('position offset between catalog and Gaia pos')
plt.xlabel('y position in pixels')
plt.ylabel('magnitude of vector diff in pixels')

In [ ]:
# Get RA and Dec for sources found in output catalog by pipeline
cat_ra = catalog_long['sky_centroid'].ra.deg
for i in range(len(cat_ra)):
    if (cat_ra[i]>180.):
        cat_ra[i] -= 360.
cat_dec = catalog_long['sky_centroid'].dec.deg

In [ ]:
# Look at position differences in RA and Dec rather than pixels

allRAdiff_cal = []
allDecdiff_cal = [] 
    
# Get coordinates with SkyCoord for each catalog
cat_miri = Table([cat_ra, cat_dec], names=('ra', 'dec'))
cat_gaia = Table([gaia_cat['RA'], gaia_cat['DEC']], names=('ra', 'dec'))
    
coord_cat_gaia = SkyCoord(ra=cat_gaia['ra'], dec=cat_gaia['dec'], unit="deg")
coord_cat_miri = SkyCoord(ra=cat_miri['ra'], dec=cat_miri['dec'], unit="deg")
ind_catmiri_catgaia, dist_2d, _ = match_coordinates_sky(coord_cat_gaia, coord_cat_miri)
    
# Find where the catalogs match
    
cat1_matched = cat_gaia[dist_2d.arcsec<0.05]
cat2_matched = cat_miri[ind_catmiri_catgaia[dist_2d.arcsec<0.05]]
    
#print(cat1_matched)
    
# Get differences in RA, Dec
ra_diff = cat2_matched['ra'] - cat1_matched['ra']
dec_diff = cat2_matched['dec'] - cat1_matched['dec'] 
    
#print(ra_diff)
    
# put differences in milliarcseconds
ra_diff = ra_diff * 3600000
dec_diff = dec_diff * 3600000
    
    
# Compare input RA, Dec to found RA, Dec
print('RA_Diff (mas)  Dec_diff (mas) pass/fail')
# Find if the differences are within the allowed 30 mas range
for i in np.arange(0,len(ra_diff)):
    #if ra_diff[i] < 30 and dec_diff[i] < 30:
    allRAdiff_cal.append(ra_diff[i])
    allDecdiff_cal.append(dec_diff[i])
    if abs(ra_diff[i]) < 30 and abs(dec_diff[i]) < 30: 
        test = 'pass' 
    else: 
        test = 'fail'
    print('{:15.6f} {:15.6f} {}'.format(ra_diff[i], dec_diff[i], test))
 
    
# Plot ra and dec differences
plt.title ('Differences in RA and Dec in milliarcseconds')
plt.ylabel('Delta RA (mas)')
plt.xlabel('Delta Dec (mas)')
plt.scatter(ra_diff,dec_diff)
plt.show()

In [ ]:
# Compare Gaia catalog to the pipeline (source counts and matches based on ra and dec positions)
# set the tolerance for differences and initialize counters

tol = 1.e-5    # Set tolerance around 30 mas (units here are in degrees; 36 mas ~ 1e-5 deg)
found_count=0
multiples_count=0
missed_count=0

# Set up array for matches
detected = np.chararray(len(gaia_cat))
#print(np.shape(detected))

for ra,dec,idx in zip(gaia_cat['RA'], gaia_cat['DEC'],range(len(gaia_cat['RA']))):

    match = np.where((np.abs(cat_ra-ra) < tol) & (np.abs(cat_dec-dec) < tol))
    #print('match', match)
    
    if np.size(match) == 1: 
        found_count +=1 
        detected[idx] = 'Y'

    if np.size(match) > 1:  
        multiples_count +=1       
        
    if np.size(match) < 1:
        missed_count +=1

In [ ]:
total_percent_found = (found_count/len(gaia_cat['RA']))*100

print('\n')
print('SNR threshold used for pipeline: ',pipe3.source_catalog.snr_threshold)
print('Total matches found:',found_count)
#print('Total missed:',missed_count)
print('Number of multiples: ',multiples_count)
print('Total number of input (Gaia) sources:',len(gaia_cat['RA']))
print('Total number in output JWST catalog:',len(cat_ra))
print('Total percent found:',total_percent_found)
print('\n')

### Look at the photometry of the two filters

In [ ]:
# Look at flux
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(catalog_long['xcentroid'], catalog_long['aper_total_flux'],color='blue', s=15, label='F2180W')
ax.scatter(catalog['xcentroid'], catalog['aper_total_flux'], color='green', s=15, label='F7700W')
ax.legend()

plt.yscale('log')
plt.title('Total Flux in '+ str(catalog_long['aper_total_flux'].unit))
plt.xlabel('xcentroid')
plt.ylabel('aper_total_flux')

In [ ]:
# See the relationship between Flux and error
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

maxflux = 0.0012

index = np.where(catalog['aper_total_flux'] < maxflux)
index_long = np.where(catalog_long['aper_total_flux'] < maxflux)

ax.scatter(catalog_long['aper_total_flux'][index_long], catalog_long['aper_total_flux_err'][index_long], color='blue', s=10, label='F2180W')
ax.scatter(catalog['aper_total_flux'][index], catalog['aper_total_flux_err'][index], color='green', s=10, label='F7700W')
ax.legend()
#plt.yscale('log')
plt.title('Total Flux vs. Flux err')
plt.xlabel('Flux in Jy')
plt.ylabel('Flux err')

In [ ]:
# Look at AB magnitudes
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

ax.scatter(catalog_long['xcentroid'], catalog_long['aper_total_abmag'], color='blue', s=15, label='F2180W')
ax.scatter(catalog['xcentroid'], catalog['aper_total_abmag'], color='green', s=15, label='F770W')
ax.legend()

#plt.yscale('log')
plt.title('Total AB mag')
plt.xlabel('xcentroid')
plt.ylabel('aper_total_abmag')

In [ ]:
# See the relationship between AB mag and error
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()

maxmag = 23

index = np.where(catalog['aper_total_abmag'] < maxmag)
index_long = np.where(catalog_long['aper_total_abmag'] < maxmag)


ax.scatter(catalog_long['aper_total_abmag'][index_long], catalog_long['aper_total_abmag_err'][index_long], color='blue', s=15, label='F2180W')
ax.scatter(catalog['aper_total_abmag'][index], catalog['aper_total_abmag_err'][index], color='green', s=15, label='F770W')
ax.legend()

#plt.yscale('log')
plt.title('Total AB mag vs. AB mag err')
plt.xlabel('AB mag')
plt.ylabel('AB mag err')

### Metrics:

This notebook passes if there are a reasonable number of close matches between the Gaia catalogs and the output positions from source catalog. The differences should show up as a gaussian centered around (0,0) in the plots of deltax v. deltay, with not too many differences above half a pixel. There will be more source matches at the F770W filter than at the F1280W filter. The photometry plots comparing the two filters should also show them being similar.

<a id="about_ID"></a>
### About this Notebook

Authors: M. Cracraft and M. Libralato: MIRI Branch

Updated On: 08/19/2022